In [25]:
import os
import pandas as pd

# Folder path
folder_path = "/home/metehan/Visual_Studio_Projects/code-gen/evaluation_files"

# Create a global 'metrics' folder in the current local directory
global_metrics_folder = "metrics"
os.makedirs(global_metrics_folder, exist_ok=True)

# Walk through all files in the folder and subfolders
for root, _, files in os.walk(folder_path):
    # Collect results for the current subfolder
    results = []
    subfolder_name = os.path.relpath(root, folder_path)  # Get the subfolder name (relative path)
    
    for file in files:
        if file.endswith(".csv"):
            file_path = os.path.join(root, file)
            file_name = os.path.relpath(file_path, folder_path)  # Get relative path for unique name
            
            try:
                # Read the file
                df = pd.read_csv(file_path)
                
                # Ensure required columns exist
                required_columns = ['numRefactoredCorrect', 'numOptimizedCorrect', 
                                    'total', 'runtimeOriginal', 'runtimeRefactored', 
                                    'runtimeOptimized', 'eslint_count_original', 
                                    'eslint_count_refactored', 'closure_count_original', 
                                    'closure_count_refactored']
                
                missing_columns = [col for col in required_columns if col not in df.columns]
                if missing_columns:
                    print(f"Skipping file {file_path}: Missing columns {missing_columns}")
                    continue

                # Separate DataFrame for runtime calculations
                runtime_df = df[
                    (df['runtimeOriginal'].between(0, 60)) &
                    (df['runtimeRefactored'].between(0, 60)) &
                    (df['runtimeOptimized'].between(0, 60))
                ]

                # If no valid rows remain for runtime calculations, skip runtime calculations
                if runtime_df.empty:
                    print(f"No valid rows for runtime calculations in file {file_path}. Skipping runtime calculations.")
                    average_runtime_improvement_refactored = None
                    average_runtime_improvement_optimized = None
                    average_runtime_comparison_percentage = None
                else:

                    # Runtime Improvements (Percentage Change)
                    runtime_df['runtime_improvement_refactored (%)'] = (
                        (runtime_df['runtimeRefactored'] - runtime_df['runtimeOriginal']) / runtime_df['runtimeOriginal']
                    ) * 100
                    runtime_df['runtime_improvement_optimized (%)'] = (
                        (runtime_df['runtimeOptimized'] - runtime_df['runtimeOriginal']) / runtime_df['runtimeOriginal']
                    ) * 100

                    # Average percentage improvements
                    average_runtime_improvement_refactored = runtime_df['runtime_improvement_refactored (%)'].mean()
                    average_runtime_improvement_optimized = runtime_df['runtime_improvement_optimized (%)'].mean()

                    # Comparison Between Refactored and Optimized
                    runtime_df['runtime_comparison (%)'] = ((1 + average_runtime_improvement_refactored/100) / (1 + average_runtime_improvement_optimized/100) - 1) * 100
                    average_runtime_comparison_percentage = runtime_df['runtime_comparison (%)'].mean()

                # Accuracy Metrics
                df['accuracy_refactored'] = df['numRefactoredCorrect'] / df['total']
                df['accuracy_optimized'] = df['numOptimizedCorrect'] / df['total']
                accuracy_change_percentage = (
                    (df['accuracy_refactored'].mean() - df['accuracy_optimized'].mean()) /
                    df['accuracy_optimized'].mean()
                ) * 100

                # Linter Issues (Percentage Change)
                df['eslint_change_percentage'] = (
                    (df['eslint_count_refactored'] - df['eslint_count_original']) /
                    df['eslint_count_original']
                ) * 100
                df['closure_change_percentage'] = (
                    (df['closure_count_refactored'] - df['closure_count_original']) /
                    df['closure_count_original']
                ) * 100

                average_eslint_change = df['eslint_change_percentage'].mean()
                average_closure_change = df['closure_change_percentage'].mean()

                # Append results for this file
                results.append({
                    "File": file_name,
                    # Accuracy
                    "Average Accuracy (Refactored)": "{:.4f}".format(df['accuracy_refactored'].mean()),
                    "Average Accuracy (Optimized)": "{:.4f}".format(df['accuracy_optimized'].mean()),
                    "Accuracy Change Percentage": "{:.4f}".format(accuracy_change_percentage),
                    # Runtime Metrics
                    "Average Runtime Improvement (Refactored %)": "{:.4f}".format(average_runtime_improvement_refactored),
                    "Average Runtime Improvement (Optimized %)": "{:.4f}".format(average_runtime_improvement_optimized),
                    "Average Runtime Comparison (Refactored vs. Optimized %)": "{:.4f}".format(average_runtime_comparison_percentage),
                    # Linter Issues
                    "Average ESLint Change Percentage": "{:.4f}".format(average_eslint_change),
                    "Average Closure Change Percentage": "{:.4f}".format(average_closure_change),
                })

            except Exception as e:
                print(f"Error processing file {file_path}: {e}")
    
    # If there are results, save them to a CSV in the global 'metrics' folder
    if results:
        output_csv = os.path.join(global_metrics_folder, f"metrics_{subfolder_name.replace(os.sep, '_')}.csv")
        results_df = pd.DataFrame(results)
        results_df.to_csv(output_csv, index=False)
        print(f"Metrics saved for subfolder '{subfolder_name}' to {output_csv}")


Metrics saved for subfolder 'result_gemini/results_gemini_0.2' to metrics/metrics_result_gemini_results_gemini_0.2.csv
Metrics saved for subfolder 'result_gemini/results_gemini_0' to metrics/metrics_result_gemini_results_gemini_0.csv
Metrics saved for subfolder 'openai_results_temp_0' to metrics/metrics_openai_results_temp_0.csv


/tmp/ipykernel_8083/2252020652.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  runtime_df['runtime_improvement_refactored (%)'] = (
/tmp/ipykernel_8083/2252020652.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  runtime_df['runtime_improvement_optimized (%)'] = (
/tmp/ipykernel_8083/2252020652.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas